In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib
import customtkinter as ctk
from tkinter import messagebox
import tkinter.ttk as ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import re
from PIL import Image  # For loading logo image

# STEP 2: Load and Combine Datasets
file1 = pd.read_csv("C:/Users/DELL/Downloads/Mill Pass Dump.csv")
file2 = pd.read_csv("C:/Users/DELL/Downloads/ROLLING_MILL_2_PASS_DATA (1) (1).csv")
combined_df = pd.concat([file1, file2], ignore_index=True)

# STEP 3: Filter Relevant Columns and Constraints
df_filtered = combined_df[["PASS_INPUT_THICKNESS", "PASS_OUTPUT_THICKNESS", "PASS_NO"]].copy()
df_filtered = df_filtered[(df_filtered["PASS_OUTPUT_THICKNESS"] < df_filtered["PASS_INPUT_THICKNESS"])]
df_filtered = df_filtered[(
    (df_filtered["PASS_INPUT_THICKNESS"] >= 0.32) & (df_filtered["PASS_INPUT_THICKNESS"] <= 5.2) &
    (df_filtered["PASS_OUTPUT_THICKNESS"] >= 0.28) & (df_filtered["PASS_OUTPUT_THICKNESS"] <= 4.75)
)]

# STEP 4: Prepare Data for ML
X = df_filtered[["PASS_OUTPUT_THICKNESS"]]
y_input = df_filtered["PASS_INPUT_THICKNESS"]
y_pass = df_filtered["PASS_NO"]

# STEP 5: Train Random Forest Models
input_model = RandomForestRegressor(n_estimators=200, random_state=42)
pass_model = RandomForestRegressor(n_estimators=200, random_state=42)
input_model.fit(X, y_input)
pass_model.fit(X, y_pass)

# STEP 6: Save Models
joblib.dump(input_model, "pass_input_model.pkl")
joblib.dump(pass_model, "pass_no_model.pkl")

# STEP 7: GUI using CustomTkinter
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

class RollingMillPredictor(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Rolling Mill Predictor - RF Model")
        self.geometry("1000x700")
        self.input_model = joblib.load("pass_input_model.pkl")
        self.pass_model = joblib.load("pass_no_model.pkl")
        self.create_widgets()

    def create_widgets(self):
        try:
            logo_image = ctk.CTkImage(Image.open("C:/Users/DELL/Downloads/tata logo.png"), size=(50, 50))
            logo_label = ctk.CTkLabel(self, image=logo_image, text="")
            logo_label.pack(pady=(10, 5), anchor="center")
        except Exception as e:
            print("Logo could not be loaded:", e)

        frame = ctk.CTkFrame(self)
        frame.pack(padx=20, pady=10, fill="x")

        title = ctk.CTkLabel(frame, text="Predict Rolling Mill Parameters", font=("Roboto", 20))
        title.pack(pady=10)

        self.entry_label = ctk.CTkLabel(frame, text="Enter PASS_OUTPUT_THICKNESS (mm):", font=("Roboto", 14))
        self.entry_label.pack(pady=5)

        self.output_entry = ctk.CTkEntry(frame, font=("Roboto", 14), width=200)
        self.output_entry.pack(pady=5)
        self.output_entry.bind("<Return>", lambda event: self.predict_thickness())

        self.predict_btn = ctk.CTkButton(frame, text="Predict", command=self.predict_thickness, font=("Roboto", 14))
        self.predict_btn.pack(pady=10)

        self.result_input = ctk.CTkLabel(frame, text="", font=("Roboto", 14))
        self.result_input.pack(pady=5)

        self.result_pass = ctk.CTkLabel(frame, text="", font=("Roboto", 14))
        self.result_pass.pack(pady=5)

        self.sequence_frame = ctk.CTkFrame(self)
        self.sequence_frame.pack(fill="both", expand=True, padx=20, pady=10)

        columns = ("Pass", "Input (mm)", "Output (mm)", "Reduction (mm)", "Reduction (%)")
        self.tree = ttk.Treeview(self.sequence_frame, columns=columns, show="headings")
        for col in columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, anchor="center", width=120)
        self.tree.pack(side="left", fill="both", expand=True)

        self.chart_frame = ctk.CTkFrame(self)
        self.chart_frame.pack(fill="both", expand=True, padx=20, pady=10)

    def generate_reduction_sequence(self, initial_thickness, target_thickness, passes):
        sequence = []
        current_thickness = initial_thickness

        total_reduction = initial_thickness - target_thickness
        base_reduction = total_reduction / passes

        reductions = [base_reduction * (1.5 - (i / passes)) for i in range(passes)]
        scaling_factor = total_reduction / sum(reductions)
        reductions = [r * scaling_factor for r in reductions]

        for i in range(passes):
            if i == passes - 1:
                next_thickness = target_thickness
            else:
                next_thickness = current_thickness - reductions[i]
                next_thickness = max(next_thickness, target_thickness)

            reduction = current_thickness - next_thickness
            reduction_percent = (reduction / current_thickness) * 100
            sequence.append({
                "pass": i + 1,
                "input": round(current_thickness, 3),
                "output": round(next_thickness, 3),
                "reduction_mm": round(reduction, 3),
                "reduction_percent": round(reduction_percent, 2)
            })
            current_thickness = next_thickness

        return sequence

    def plot_reduction_sequence(self, sequence):
        for widget in self.chart_frame.winfo_children():
            widget.destroy()

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

        passes = [s['pass'] for s in sequence]
        thicknesses = [s['input'] for s in sequence] + [sequence[-1]['output']]
        reductions = [s['reduction_percent'] for s in sequence]

        ax1.scatter(passes, thicknesses[:-1], color='green', label='Input Thickness')
        ax1.plot(passes, thicknesses[:-1], color='green', linestyle='--')
        ax1.set_title('Thickness vs Pass')
        ax1.set_xlabel('Pass')
        ax1.set_ylabel('Thickness (mm)')
        ax1.grid(True)

        ax2.bar(passes, reductions, color='orange')
        ax2.set_title('Reduction % per Pass')
        ax2.set_xlabel('Pass')
        ax2.set_ylabel('Reduction (%)')
        ax2.grid(True)

        fig.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=self.chart_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill="both", expand=True)

    def predict_thickness(self):
        try:
            output_str = self.output_entry.get()
            if not re.fullmatch(r"\d*\.?\d+", output_str):
                raise ValueError("Only numeric values allowed. No alphabets or symbols.")

            output_val = float(output_str)
            if output_val < 0.28 or output_val > 4.75:
                raise ValueError("Value out of valid range (0.28 - 4.75 mm)")

            input_df = pd.DataFrame({"PASS_OUTPUT_THICKNESS": [output_val]})
            predicted_input = self.input_model.predict(input_df)[0]
            predicted_pass = max(1, int(round(self.pass_model.predict(input_df)[0])))

            self.result_input.configure(text=f"Predicted PASS_INPUT_THICKNESS: {predicted_input:.3f} mm")
            self.result_pass.configure(text=f"Estimated PASS_NO: {predicted_pass}")

            sequence = self.generate_reduction_sequence(predicted_input, output_val, predicted_pass)

            self.tree.delete(*self.tree.get_children())
            for step in sequence:
                self.tree.insert('', 'end', values=(
                    step['pass'], f"{step['input']:.3f}", f"{step['output']:.3f}",
                    f"{step['reduction_mm']:.3f}", f"{step['reduction_percent']:.2f}"
                ))

            self.plot_reduction_sequence(sequence)

        except ValueError as ve:
            messagebox.showerror("Invalid Input", str(ve))
        except Exception as e:
            messagebox.showerror("Error", f"Something went wrong: {str(e)}")

if __name__ == "__main__":
    app = RollingMillPredictor()
    app.mainloop()
